# Why GT4Py? (Part I)

This notebook compares a NumPy, CuPy and GT4Py implementation of the point-wise stencil
```
d[i, j, k] = a[i, j, k] + b[i, j, k] - c[i, j, k]
```
Timings are measured using the `%timeit` magic command. This command times a single statement or function call by averaging over multiple runs. It has the additional advantage of synchronizing the CPU and the GPU at the end of each run, thus making the output reliable.

## NumPy

In [ ]:
import numpy as np

shape = (512, 512, 128)

def f_numpy(a, b, c, d):
    d[...] = a + b - c
    
a = np.random.rand(*shape)
b = np.random.rand(*shape)
c = np.random.rand(*shape)
d = np.empty_like(a)

%timeit f_numpy(a, b, c, d)

## CuPy

In [ ]:
import cupy as cp

shape = (512, 512, 128)

def f_cupy(a, b, c, d):
    d[...] = a + b - c
    
a = cp.random.rand(*shape)
b = cp.random.rand(*shape)
c = cp.random.rand(*shape)
d = cp.empty_like(a)

%timeit f_cupy(a, b, c, d)

## GT4Py

In [ ]:
import gt4py as gt
from gt4py import gtscript
import numpy as np

backend = "gtcuda"
shape = (512, 512, 128)

@gtscript.stencil(backend=backend, verbose=True)
def f_gt4py(
    a: gtscript.Field[float], 
    b: gtscript.Field[float], 
    c: gtscript.Field[float], 
    d: gtscript.Field[float]
):
    with computation(PARALLEL), interval(...):
        d = a + b - c
        
a_np = np.random.rand(*shape)
b_np = np.random.rand(*shape)
c_np = np.random.rand(*shape)

a = gt.storage.from_array(a_np, backend, (0, 0, 0))
b = gt.storage.from_array(b_np, backend, (0, 0, 0))
c = gt.storage.from_array(c_np, backend, (0, 0, 0))
d = gt.storage.empty(backend, (0, 0, 0), shape, float)

# copy data to GPU memory
f_gt4py(a=a, b=b, c=c, d=d, origin=(0, 0, 0), domain=shape)

%timeit f_gt4py(a=a, b=b, c=c, d=d, origin=(0, 0, 0), domain=shape)